# Imports

In [3]:
# Data manipulation
from collections import Counter
from unidecode import unidecode
from datetime import datetime as dt
import re
import numpy as np
import pandas as pd

# NPL
import nltk
import spacy
import emoji

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from wordcloud import WordCloud

# Progress bar
from tqdm import tqdm; tqdm.pandas();

# Plot colors
URBAN_PALETTE = pd.DataFrame({
    'cyan'    : '#1696d2',
    'gray'    : '#d2d2d2',
    'black'   : '#000000',
    'yellow'  : '#fdbf11',
    'magenta' : '#ec008b',
    'green'   : '#55b748',
    'red'     : '#db2b27',
}, index=['hex_code'])

import session_info

session_info.show()

Python 3.9.7


# Read data

In [38]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)

In [5]:
!ls -la data

total 248792
drwxr-xr-x  5 baiochi  staff        160 May 24 21:02 .
drwxr-xr-x  6 baiochi  staff        192 May 24 21:05 ..
-rw-rw-r--@ 1 baiochi  staff  111753484 May 24 04:24 20220524_Pedidos_csv_2022.csv
-rw-rw-r--@ 1 baiochi  staff   14264754 May 24 04:26 20220524_Recursos_csv_2022.csv
-rw-rw-r--@ 1 baiochi  staff    1359586 May 24 04:26 20220524_Solicitantes_csv_2022.csv


In [28]:
solicitantes_df = pd.read_csv('data/20220524_Solicitantes_csv_2022.csv', encoding='utf-16', sep=';', header=None)
solicitantes_df.head()

,0,1,2,3,4,5,6,7,8,9
0,11,Pessoa Física,,Masculino,,,,,DISTRITO FEDERAL,Brasília
1,1437,Pessoa Física,,Masculino,,,,,ALAGOAS,Feira Grande
2,2782,Pessoa Física,,Masculino,,,,,PARANÁ,Almirante Tamandaré
3,6065,Pessoa Física,,Masculino,,,,,PARANÁ,Porto Amazonas
4,6329,Pessoa Física,,Masculino,Pós-graduação,Profis. Liberal/autônomo,,Brasil,BAHIA,Salvador


In [29]:
requests_df = pd.read_csv('data/20220524_Pedidos_csv_2022.csv', encoding='utf-16', sep=';', header=None)
requests_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,3878674,60141000001202295,Federal,COMAER – Comando da Aeronáutica,Concluída,01/01/2022,Porque a FAB é chamada de Comando da Aeronaútica do DOU e não Força Aerea Brasileira?,Porque a FAB é chamada de Comando da Aeronaútica do DOU e não Força Aerea Brasileira?,03/02/2022,Sim,Não,Pelo sistema (com avisos por email),Internet,1145978,Acesso à informação,,,02/02/2022,"MINISTÉRIO DA DEFESA COMANDO DA AERONÁUTICA Centro de Comunicação Social da Aeronáutica Senhor Lázaro, Sobre a solicitação em comento, inicialmente, cumpre-nos informar que o canal utilizado não se presta para esclarecimentos de dúvidas, interpretação de normas/legislação, nem de providências para os órgãos públicos. Nada obstante, sugerimos que leia as normas de criação do, então, Ministério da Aeronáutica, Decreto-Lei nº 2.961, de 20 de janeiro de 1941, sua transformação em Comando da Aeronáutica, Medida Provisória nº 2.216-37, de 31 de agosto de 2001, e da Força Aérea Brasileira, Decreto-Lei nº 3.302, de 22 de maio de 1941. As legislações referenciadas podem ser obtidas nos seguintes links: http://www.planalto.gov.br/ccivil_03/mpv/2216-37.htm http://www.planalto.gov.br/CCiVil_03/Decreto-Lei/1937-1946/Del2961.htm https://www2.camara.leg.br/legin/fed/declei/1940-1949/decreto-lei-3302-22-maio-1941-413224-publicacaooriginal-1-pe.html Serviço de Informações ao Cidadão Centro de Comunicação Social da Aeronáutica Esplanada dos Ministérios – Bloco M - 7º andar – Brasília – Distrito Federal CEP 70.045-900.",Não se trata de solicitação de informação,
1,3878729,23546000005202294,Federal,UFMS – Fundação Universidade Federal de Mato Grosso do Sul,Concluída,01/01/2022,Solicita Informações Telefones Celulares Corporativos (Funcionais),"Em atenção aos enunciados dos art. 6º, VI, e art. 10 da Lei de Acesso à Informação Art. 6º Cabe aos órgãos e entidades do poder público, observadas as normas e procedimentos específicos aplicáveis, assegurar a: VI - informação pertinente à administração do patrimônio público, utilização de recursos públicos, licitação, contratos administrativos, e Art. 10. Qualquer interessado poderá apresentar pedido de acesso a informações aos órgãos e entidades referidos no art. 1º desta Lei, por qualquer meio legítimo, devendo o pedido conter a identificação do requerente e a especificação da informação requerida. Solicito uma lista, com o nome completo e lotação, de todos os servidores técnicos e docentes da UFMS que estão em posse de celulares corporativos (funcionais). Ressalto que não me interessa o número dos respectivos aparelhos, apenas quem são os agentes públicos da UFMS e em quais lugares atuam na UFMS. Ressalto, ainda, que a informação solicitada não está abrangida por qualquer restrição constitucional existente como o sigilo bancário ou fiscal. Att.,",24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,1295308,Acesso à informação,Informação sobre servidor(es),Telefone(s) Celular(es) Funcional(is),11/01/2022,"Boa tarde! Agradecemos por nos contatar. Informamos que sua demanda foi analisada pela Ouvidoria e encaminhada à área competente, Gabinte da Pró-Reitoria de Administração e Infraestrutura, que nos enviou a seguinte resposta: __________________________________ Bom dia, Seguem as informações solicitadas sobre as linhas ativas e desbloqueadas no momento: Camila Celeste Brandao Ferreira Itavo - Reitoria Cleuir Viegas da Silva Mascarenhas - Rádio Educativa UFMS 99.9 Jamil Farrhat Valentim - Reitoria Luciano Gonda - Agetic Marcelo Augusto Santos Turine - Reitoria Nilton Conde Torres - Diserv/Proadi Bianca Berigo Ferreira - Seman/Dinfra/Proadi Informamos que há também 16 linhas ativas que compõem o sistema PABX da UFMS. __________________________________ Sem mais, colocamo-nos a sua disposição e agradecemos novamente pelo contato com esta Ouvidoria. Participe da nossa pesquisa de satisfação! Avaliar o trabalho desta Ouvidoria ajuda-nos a melhorar nosso atendimento! Atenciosamente, Ouvid

In [30]:
resources_df = pd.read_csv('data/20220524_Recursos_csv_2022.csv', encoding='utf-16', sep=';', header=None)
resources_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,152195,,"Prezados(as), Agradeço o retorno, porém a resposta quanto à segunda pergunta não parece corresponder à realidade sobretudo no caso de Roraima e com processos minerários com áreas sobrepostas à Terras Indígenas. Dada a resposta ""Resposta: Os processos irão à disponibilidade, não tem outra forma, isto é determinante."" Pois, ao contrário do que ocorre nas Terras Indígenas do Pará, em que os processos abertos nas Terras Indígenas do respectivo estado, são constantemente indeferidos e inativados, nas Terras Indígenas de Roraima, prevalecem ativos processos minerários anteriores à 1990, ou seja, anterior à homologação da TI Yanomami, por exemplo, e também posteriores à homologação em 1992. Então aqui seguem as perguntas sobre esses casos específicos: A) Os processos minerários com sobreposição à Terras Indígenas, mesmo que protocolados anteriormente a homologação da mesma, já não deveriam constar como inativos? Visto que, em sua maioria, os processos minerários ativos e sobrepostos às Terras Indígenas de RR possuem em comum os últimos eventos: -Código 111: Req Pesquisa/Consulta FUNAI Encaminhada, - Código 619: Req Pesq/Processo Situado em Área Indígena (a maioria com os eventos registrados antes de 2005), - Código 157: Req Pesq/Desistência Req Pesq Homologada Pub (a maioria com os eventos registrados antes de 2018), - Código 136: Req Pesq/Documento Diverso Protocolizado. Reforço a pergunta, dado que a maioria desses processos ATIVOS que sobrepõem Terras Indígenas em RR, concentram como últimos eventos por exemplo a situação locacional (que não regulamenta a mineração em Terras Indígenas) e a desistência do requerimento de pesquisa antes mesmo de obter o título, por que esses processos permanecem ativos mesmo com o último evento ocorrendo por exemplo em junho de 2005. B) Caso a regulamentação da mineração em Terras Indígenas seja aprovada, esses processos que sobrepõem as Terras Indígenas em Roraima e permanecem ativos ainda hoje, terão prioridade, na área do respectivo processo, para exercer a atividade de pesquisa requisitada originalmente (sendo a maioria protocolado em 1984 e que continuam ativo)? Ou seja, mesmo com a desistência do requerimento já homologada, mesmo com a situação locacional hoje definida como não regulamentar, a situação do processo constar como ATIVO, pode privilegiar, pelo princípio da prioridade, os responsáveis legais por determinados processos a exercerem, de forma prioritária, a atividade mediante regulamentação da atividade, ou novos processos deverão ser protocolados?",3880026,0,48003000013202207,ANM - Agência Nacional de Mineração,Primeira Instância,Respondido,03/01/2022,10/01/2022,Internet,Informação incompleta,03/01/2022,"A) Os processos minerários com sobreposição à Terras Indígenas, mesmo que protocolados anteriormente a homologação da mesma, já não deveriam constar como inativos? Visto que, em sua maioria, os processos minerários ativos e sobrepostos às Terras Indígenas de RR possuem em comum os últimos eventos: -Código 111: Req Pesquisa/Consulta FUNAI Encaminhada, - Código 619: Req Pesq/Processo Situado em Área Indígena (a maioria com os eventos registrados antes de 2005), - Código 157: Req Pesq/Desistência Req Pesq Homologada Pub (a maioria com os eventos registrados antes de 2018), - Código 136: Req Pesq/Documento Diverso Protocolizado. Reforço a pergunta, dado que a maioria desses processos ATIVOS que sobrepõem Terras Indígenas em RR, concentram como últimos eventos por exemplo a situação locacional (que não regulamenta a mineração em Terras Indígenas) e a desistência do requerimento de pesquisa antes mesmo de obter o título, por que esses processos permanecem ativos mesmo com o último evento ocorrendo por exemplo em junho de 2005. Resposta: Processo em Terra Indígena não vai à Disponibilidade, a resposta inicial foi em função da pergunta que Vossa Senhoria fez, não identificou que seria em Terra Indígena, concluindo, nenhum processo minerário

In [31]:
requests_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,3878674,60141000001202295,Federal,COMAER – Comando da Aeronáutica,Concluída,01/01/2022,Porque a FAB é chamada de Comando da Aeronaútica do DOU e não Força Aerea Brasileira?,Porque a FAB é chamada de Comando da Aeronaútica do DOU e não Força Aerea Brasileira?,03/02/2022,Sim,Não,Pelo sistema (com avisos por email),Internet,1145978,Acesso à informação,,,02/02/2022,"MINISTÉRIO DA DEFESA COMANDO DA AERONÁUTICA Centro de Comunicação Social da Aeronáutica Senhor Lázaro, Sobre a solicitação em comento, inicialmente, cumpre-nos informar que o canal utilizado não se presta para esclarecimentos de dúvidas, interpretação de normas/legislação, nem de providências para os órgãos públicos. Nada obstante, sugerimos que leia as normas de criação do, então, Ministério da Aeronáutica, Decreto-Lei nº 2.961, de 20 de janeiro de 1941, sua transformação em Comando da Aeronáutica, Medida Provisória nº 2.216-37, de 31 de agosto de 2001, e da Força Aérea Brasileira, Decreto-Lei nº 3.302, de 22 de maio de 1941. As legislações referenciadas podem ser obtidas nos seguintes links: http://www.planalto.gov.br/ccivil_03/mpv/2216-37.htm http://www.planalto.gov.br/CCiVil_03/Decreto-Lei/1937-1946/Del2961.htm https://www2.camara.leg.br/legin/fed/declei/1940-1949/decreto-lei-3302-22-maio-1941-413224-publicacaooriginal-1-pe.html Serviço de Informações ao Cidadão Centro de Comunicação Social da Aeronáutica Esplanada dos Ministérios – Bloco M - 7º andar – Brasília – Distrito Federal CEP 70.045-900.",Não se trata de solicitação de informação,
1,3878729,23546000005202294,Federal,UFMS – Fundação Universidade Federal de Mato Grosso do Sul,Concluída,01/01/2022,Solicita Informações Telefones Celulares Corporativos (Funcionais),"Em atenção aos enunciados dos art. 6º, VI, e art. 10 da Lei de Acesso à Informação Art. 6º Cabe aos órgãos e entidades do poder público, observadas as normas e procedimentos específicos aplicáveis, assegurar a: VI - informação pertinente à administração do patrimônio público, utilização de recursos públicos, licitação, contratos administrativos, e Art. 10. Qualquer interessado poderá apresentar pedido de acesso a informações aos órgãos e entidades referidos no art. 1º desta Lei, por qualquer meio legítimo, devendo o pedido conter a identificação do requerente e a especificação da informação requerida. Solicito uma lista, com o nome completo e lotação, de todos os servidores técnicos e docentes da UFMS que estão em posse de celulares corporativos (funcionais). Ressalto que não me interessa o número dos respectivos aparelhos, apenas quem são os agentes públicos da UFMS e em quais lugares atuam na UFMS. Ressalto, ainda, que a informação solicitada não está abrangida por qualquer restrição constitucional existente como o sigilo bancário ou fiscal. Att.,",24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,1295308,Acesso à informação,Informação sobre servidor(es),Telefone(s) Celular(es) Funcional(is),11/01/2022,"Boa tarde! Agradecemos por nos contatar. Informamos que sua demanda foi analisada pela Ouvidoria e encaminhada à área competente, Gabinte da Pró-Reitoria de Administração e Infraestrutura, que nos enviou a seguinte resposta: __________________________________ Bom dia, Seguem as informações solicitadas sobre as linhas ativas e desbloqueadas no momento: Camila Celeste Brandao Ferreira Itavo - Reitoria Cleuir Viegas da Silva Mascarenhas - Rádio Educativa UFMS 99.9 Jamil Farrhat Valentim - Reitoria Luciano Gonda - Agetic Marcelo Augusto Santos Turine - Reitoria Nilton Conde Torres - Diserv/Proadi Bianca Berigo Ferreira - Seman/Dinfra/Proadi Informamos que há também 16 linhas ativas que compõem o sistema PABX da UFMS. __________________________________ Sem mais, colocamo-nos a sua disposição e agradecemos novamente pelo contato com esta Ouvidoria. Participe da nossa pesquisa de satisfação! Avaliar o trabalho desta Ouvidoria ajuda-nos a melhorar nosso atendimento! Atenciosamente, Ouvid

In [36]:
requests_df.columns= ['IdPedido',
'ProtocoloPedido',
'UF', 
'OrgaoDestinatario',
'Situacao',
'DataRegistro',
'Titulo',
'Pergunta',
'DataResposta',
'Bool1', 
'Bool2',
'Sistema',
'MeioComunicacao',
'OutroId',
'TipoReq1',
'TipoReq2',
'TipoReq3',
'UltimaResposta',
'Decisao',
'EspecificacaoDecisao',
'RespFinal']

In [39]:
requests_df.head()

,IdPedido,ProtocoloPedido,UF,OrgaoDestinatario,Situacao,DataRegistro,Titulo,Pergunta,DataResposta,Bool1,Bool2,Sistema,MeioComunicacao,OutroId,TipoReq1,TipoReq2,TipoReq3,UltimaResposta,Decisao,EspecificacaoDecisao,RespFinal
0,3878674,60141000001202295,Federal,COMAER – Comando da Aeronáutica,Concluída,01/01/2022,Porque a FAB é chamada de Comando da Aeronaútica do DOU e não Força Aerea Brasileira?,Porque a FAB é chamada de Comando da Aeronaútica do DOU e não Força Aerea Brasileira?,03/02/2022,Sim,Não,Pelo sistema (com avisos por email),Internet,1145978,Acesso à informação,,,02/02/2022,MINISTÉRIO DA DEFESA COMANDO DA AERONÁUTICA Centro de Comunicação Social da Aeronáutica Sen...,Não se trata de solicitação de informação,
1,3878729,23546000005202294,Federal,UFMS – Fundação Universidade Federal de Mato Grosso do Sul,Concluída,01/01/2022,Solicita Informações Telefones Celulares Corporativos (Funcionais),"Em atenção aos enunciados dos art. 6º, VI, e art. 10 da Lei de Acesso à Informação Art. 6º Ca...",24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,1295308,Acesso à informação,Informação sobre servidor(es),Telefone(s) Celular(es) Funcional(is),11/01/2022,Boa tarde! Agradecemos por nos contatar. Informamos que sua demanda foi analisada pela Ouvido...,Acesso Concedido,Resposta solicitada inserida no Fala.Br
2,3878738,23546000009202272,Federal,UFCA - Universidade Federal do Cariri,Concluída,01/01/2022,SOLICITA INFORMAÇÕES,"Prezados Senhores, Considerando a Lei de Acesso à Informação em seus Art. 6º e 10º, venho sol...",24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,4251991,Cotas,,,18/01/2022,"Boa tarde! Conforme solicitado, segue respostas . 1 – Quantas/os professoras/es negras/os...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
3,3878739,23546000010202205,Federal,UNIFEI – Universidade Federal de Itajubá,Concluída,01/01/2022,SOLICITA INFORMAÇÕES,"Prezados Senhores, Considerando a Lei de Acesso à Informação em seus Art. 6º e 10º, venho sol...",24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,4251991,Cotas,,,11/01/2022,"Prezado VANTUIL PEREIRA, Bom dia! Segue em anexo sua resposta NUP 23546.000010/2022-05 ...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
4,3878740,23546000011202241,Federal,UNIFAL-MG – Universidade Federal de Alfenas,Concluída,01/01/2022,SOLICITA INFORMAÇÕES,"Prezados Senhores, Considerando a Lei de Acesso à Informação em seus Art. 6º e 10º, venho sol...",24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,4251991,Outros em Trabalho,,,18/01/2022,Prezado solicitante. Espero que esteja bem. Em atendimento ao seu pedido de acesso à inf...,Acesso Concedido,Resposta solicitada inserida no Fala.Br


In [41]:
df[df['Pergunta'].str.contains("bolsonaro")]

,IdPedido,ProtocoloPedido,UF,OrgaoDestinatario,Situacao,DataRegistro,Titulo,Pergunta,DataResposta,Bool1,Bool2,Sistema,MeioComunicacao,OutroId,TipoReq1,TipoReq2,TipoReq3,UltimaResposta,Decisao,EspecificacaoDecisao,RespFinal
56,3879448,23546000069202295,Federal,FNDE – Fundo Nacional de Desenvolvimento da Educação,Concluída,02/01/2022,info,O presidente editou uma MP que permite renegociar a dívida do FIES - https://www.cnnbrasil.com.b...,24/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,1309345,Acesso à informação,,,20/01/2022,"Prezado (a) Senhor (a), Encaminhamos-lhe, em anexo, resposta referente ao pedido SIC 23...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
57,3879449,23546000070202210,Federal,FNDE – Fundo Nacional de Desenvolvimento da Educação,Concluída,02/01/2022,info,O presidente editou uma MP que permite renegociar a dívida do FIES - https://www.cnnbrasil.com.b...,24/01/2022,Sim,Sim,Pelo sistema (com avisos por email),Internet,1309345,Acesso à informação,,,04/01/2022,"Prezado (a) Senhor (a), Informamos que o seu pedido SIC 23546.000070/2022-10, será aten...",Pergunta Duplicada/Repetida,
637,3890304,60110000034202201,Federal,CMAR – Comando da Marinha,Concluída,05/01/2022,Info,Considerando que existe legislação específica desde 2012 que veta certos tipos de tatuagem em mi...,26/01/2022,Sim,Sim,Pelo sistema (com avisos por email),Internet,1309345,Acesso à informação,,,24/01/2022,"Prezado Senhor Luiz Fernando Toledo, Em atenção a sua manifestação, este Serviço participa q...",Informação Inexistente,
638,3890309,60143000083202258,Federal,CEX – Comando do Exército,Concluída,05/01/2022,Info,Considerando que existe legislação específica desde 2012 que veta certos tipos de tatuagem em mi...,04/02/2022,Sim,Não,Pelo sistema (com avisos por email),Internet,1309345,Exército,,,04/02/2022,"Prezado Senhor, Ao cumprimentá-lo, cordialmente, o Serviço de Informações ao Cidadão do Exér...",Informação Inexistente,
639,3890312,60000000026202201,Federal,CMAR – Comando da Marinha,Concluída,05/01/2022,Info,Considerando que existe legislação específica desde 2012 que veta certos tipos de tatuagem em mi...,25/01/2022,Não,Não,Pelo sistema (com avisos por email),Internet,1309345,Acesso à informação,,,24/01/2022,"Prezado Senhor Luiz Fernando Toledo, Agradecemos o seu contato com a Marinha do Brasil pela p...",Informação Inexistente,
640,3890315,60141000030202257,Federal,COMAER – Comando da Aeronáutica,Concluída,05/01/2022,Info,Considerando que existe legislação específica desde 2012 que veta certos tipos de tatuagem em mi...,04/02/2022,Sim,Não,Pelo sistema (com avisos por email),Internet,1309345,Acesso à informação,,,04/02/2022,MINISTÉRIO DA DEFESA COMANDO DA AERONÁUTICA Centro de Comunicação Social da Aeronáutica Sen...,Acesso Concedido,Resposta solicitada inserida no Fala.Br
2568,3930965,60110000156202299,Federal,MD – Ministério da Defesa,Concluída,17/01/2022,eleições,"Considerando a fala abaixo do presidente da República, envio alguns questionamentos https://w...",17/02/2022,Sim,Não,Pelo sistema (com avisos por email),Internet,2162000,Acesso à informação,,,17/02/2022,"Prezado(a) Cidadão(ã), Ao cumprimentá-lo(a) cordialmente, reporto-me ao pedido formulado por ...",Acesso Negado,Processo decisório em curso
3244,3943216,137000818202209,Federal,GSI-PR – Gabinete de Segurança Institucional da Presidência da República,Concluída,19/01/2022,documentos,Com fundamento na Lei 12.527/2011 (Lei de Acesso a Informações Públicas) venho requerer o acesso...,09/02/2022,Não,Não,Pelo sistema (com avisos por email),Internet,2296256,Acesso à informação,,,09/02/2022,"Prezado Cidadão, Em atenção ao pedido de acesso à informação, cadastrado sob o NUP: 00137.00081...",Informação Inexistente,
5061,3975139,137001152202206,Federal,SGPR – Secretaria-Geral da Presidência da República,Concluída,27/01/2022,documentos,Com fundamento na Lei 12.527/2011 (Lei de Acesso a Informações Públicas) venho requerer o acesso...,21/02/2022,Não,Não,Pelo sistema (com